In [129]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [130]:
batch_size = 64
num_epochs = 15 
learning_rate = 0.001
data_dir = 'tomato'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [131]:
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(128),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.3),
    transforms.RandomVerticalFlip(),
    transforms.RandomAffine(degrees=20, translate=(0.2, 0.2)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_val = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [132]:
train_dataset = datasets.ImageFolder(root='tomato/train', transform=transform_train)
val_dataset = datasets.ImageFolder(root='tomato/val', transform=transform_val)

In [133]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [134]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(128 * 16 * 16, 512)  
        self.fc2 = nn.Linear(512, len(train_dataset.classes)) 

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 16 * 16) 
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [135]:
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.7)

In [136]:
def train_model(model, train_loader, criterion, optimizer, scheduler, num_epochs=25):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_predictions = 0
        total_samples = 0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            _, predicted = torch.max(outputs, 1)
            
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()
        
        scheduler.step()
        
        average_loss = running_loss / len(train_loader)
        accuracy = (correct_predictions / total_samples) * 100
        
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {average_loss:.4f}, Accuracy: {accuracy:.2f}%')

train_model(model, train_loader, criterion, optimizer, scheduler, num_epochs=num_epochs)

Epoch 1/15, Loss: 2.0711, Accuracy: 23.26%
Epoch 2/15, Loss: 1.8061, Accuracy: 34.67%
Epoch 3/15, Loss: 1.6814, Accuracy: 39.34%
Epoch 4/15, Loss: 1.6106, Accuracy: 42.03%
Epoch 5/15, Loss: 1.5204, Accuracy: 46.37%
Epoch 6/15, Loss: 1.4240, Accuracy: 49.40%
Epoch 7/15, Loss: 1.3872, Accuracy: 50.63%
Epoch 8/15, Loss: 1.3548, Accuracy: 51.67%
Epoch 9/15, Loss: 1.3047, Accuracy: 53.55%
Epoch 10/15, Loss: 1.2711, Accuracy: 54.46%
Epoch 11/15, Loss: 1.2277, Accuracy: 57.29%
Epoch 12/15, Loss: 1.1957, Accuracy: 58.03%
Epoch 13/15, Loss: 1.1607, Accuracy: 58.84%
Epoch 14/15, Loss: 1.1514, Accuracy: 59.21%
Epoch 15/15, Loss: 1.1344, Accuracy: 59.94%


In [138]:
def evaluate_model(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Validation Accuracy: {accuracy:.2f}%')

evaluate_model(model, val_loader)

Validation Accuracy: 51.10%
